In [2]:
from neo4j import GraphDatabase

In [4]:
data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "twitch"))
session = data_base_connection.session()

In [ ]:
query = ''''''
session.run(query)